# Quinto notebook

En el siguiente notebook se  registrara de forma ordenada los experimentos de entrenamiento
del modelo YOLO usando MLflow.

Este notebook:
- Configura MLflow en modo local
- Registra parámetros del experimento
- Registra métricas de evaluación (mAP, precision, recall)
- Guarda artefactos importantes (modelo, data.yaml)
- Deja listo el historial para comparar futuros entrenamientos


In [1]:
# --- IMPORTS ---
from pathlib import Path
import json
import shutil


In [2]:
# --- RUTAS DEL PROYECTO ---
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

MODELS_DIR = PROJECT_ROOT / "models"
RUNS_DIR = PROJECT_ROOT / "runs"
ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
MLRUNS_DIR = PROJECT_ROOT / "mlruns"

CONFIG_SNAPSHOT_PATH = ARTIFACTS_DIR / "config_snapshot.json"
CLASS_MAP_PATH = ARTIFACTS_DIR / "class_map.json"

YOLO_RUN_DIR = RUNS_DIR / "yolo_coco_subset"
BEST_MODEL_PATH = MODELS_DIR / "yolo_best.pt"

print("MLRUNS_DIR:", MLRUNS_DIR)


MLRUNS_DIR: c:\Users\Johnny\Desktop\IA\mlruns


In [3]:
# --- VALIDACIONES ---
def assert_exists(p: Path, desc: str) -> None:
    if not p.exists():
        raise FileNotFoundError(f"Falta {desc}: {p}")

assert_exists(BEST_MODEL_PATH, "modelo entrenado yolo_best.pt")
assert_exists(CONFIG_SNAPSHOT_PATH, "config_snapshot.json")
assert_exists(CLASS_MAP_PATH, "class_map.json")

print("Validaciones OK.")


Validaciones OK.


In [4]:
# --- CONFIGURACIÓN DE MLFLOW ---
import mlflow

MLFLOW_TRACKING_URI = f"file:{MLRUNS_DIR.as_posix()}"
MLFLOW_EXPERIMENT_NAME = "coco2017_car_airplane_truck"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

print("Tracking URI:", MLFLOW_TRACKING_URI)
print("Experiment:", MLFLOW_EXPERIMENT_NAME)


Tracking URI: file:c:/Users/Johnny/Desktop/IA/mlruns
Experiment: coco2017_car_airplane_truck


c:\Users\Johnny\AppData\Local\Python\pythoncore-3.12-64\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)


In [5]:
# --- CARGA DE CONFIGURACIÓN ---
with open(CONFIG_SNAPSHOT_PATH, "r", encoding="utf-8") as f:
    cfg = json.load(f)

with open(CLASS_MAP_PATH, "r", encoding="utf-8") as f:
    class_map = json.load(f)

cfg, class_map


({'target_classes': ['car', 'airplane', 'truck'],
  'max_images_per_class_train': 1200,
  'max_images_per_class_val': 250,
  'img_size': 640,
  'batch_size': 16,
  'epochs': 20,
  'seed': 42,
  'conf_threshold': 0.25,
  'iou_threshold': 0.5},
 {'0': 'car', '1': 'airplane', '2': 'truck'})

In [6]:
# YOLO guarda resultados en results.csv dentro del run
# --- DETECTAR AUTOMÁTICAMENTE EL ÚLTIMO RUN DE YOLO ---
from pathlib import Path

RUNS_DIR = PROJECT_ROOT / "runs"

candidates = sorted(
    [p for p in RUNS_DIR.glob("yolo_coco_subset*") if p.is_dir()],
    key=lambda p: p.stat().st_mtime,
    reverse=True
)

if not candidates:
    raise FileNotFoundError(f"No se encontraron carpetas run tipo yolo_coco_subset* en {RUNS_DIR}")

YOLO_RUN_DIR = candidates[0]
print("YOLO_RUN_DIR detectado:", YOLO_RUN_DIR)

RESULTS_CSV = YOLO_RUN_DIR / "results.csv"
if not RESULTS_CSV.exists():
    raise FileNotFoundError(f"No se encontró results.csv en: {YOLO_RUN_DIR}")


YOLO_RUN_DIR detectado: c:\Users\Johnny\Desktop\IA\runs\yolo_coco_subset7


In [7]:
# --- CONSTRUIR METRICS SIEMPRE (DESDE results.csv) ---
import csv

RESULTS_CSV = YOLO_RUN_DIR / "results.csv"
if not RESULTS_CSV.exists():
    raise FileNotFoundError(f"No se encontró results.csv en: {YOLO_RUN_DIR}")

with open(RESULTS_CSV, newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    rows = list(reader)
    if len(rows) == 0:
        raise ValueError(f"results.csv está vacío: {RESULTS_CSV}")
    last_row = rows[-1]

metrics = {
    "precision": float(last_row.get("metrics/precision(B)", 0) or 0),
    "recall": float(last_row.get("metrics/recall(B)", 0) or 0),
    "mAP50": float(last_row.get("metrics/mAP50(B)", 0) or 0),
    "mAP50_95": float(last_row.get("metrics/mAP50-95(B)", 0) or 0),
}

metrics


{'precision': 0.44697,
 'recall': 0.37148,
 'mAP50': 0.36342,
 'mAP50_95': 0.20881}

In [8]:
# --- REGISTRO DEL EXPERIMENTO EN MLFLOW ---
with mlflow.start_run(run_name="yolo_coco_subset_limited"):

    # Parámetros
    mlflow.log_params({
        "model_type": "YOLOv8",
        "base_model": "yolov8n.pt",
        "img_size": cfg["img_size"],
        "batch_size": cfg["batch_size"],
        "epochs": cfg["epochs"],
        "optimizer": "SGD",
        "dataset_mode": "limited",
        "classes": ",".join(cfg["target_classes"])
    })

    # Métricas
    mlflow.log_metrics(metrics)

    # Artefactos
    mlflow.log_artifact(BEST_MODEL_PATH, artifact_path="model")
    mlflow.log_artifact(CONFIG_SNAPSHOT_PATH, artifact_path="config")
    mlflow.log_artifact(CLASS_MAP_PATH, artifact_path="config")

print("Experimento registrado en MLflow.")


Experimento registrado en MLflow.


In [9]:
# --- REGISTRO DE ARTEFACTOS ADICIONALES (OPCIONAL) ---
if YOLO_RUN_DIR.exists():
    mlflow.log_artifacts(YOLO_RUN_DIR, artifact_path="training_outputs")
    print("Artefactos de entrenamiento guardados.")


Artefactos de entrenamiento guardados.


In [10]:
# --- RESUMEN FINAL ---
print("MLflow configurado correctamente.")
print("Modelo registrado:", BEST_MODEL_PATH)
print("Métricas registradas:")
for k, v in metrics.items():
    print("-", k, ":", v)

MLflow configurado correctamente.
Modelo registrado: c:\Users\Johnny\Desktop\IA\models\yolo_best.pt
Métricas registradas:
- precision : 0.44697
- recall : 0.37148
- mAP50 : 0.36342
- mAP50_95 : 0.20881
